In [15]:
# === CRITICAL: Python 3.13 + NumPy 2.x Compatibility Patch ===
import random
import numpy as np
from unittest.mock import patch

def _patched_choice(population, k, replace=True):
    try:
        state = random.getstate()
        random.seed(np.random.randint(0, 2**31 - 1))
        if replace:
            return random.choices(population, k=k)
        else:
            return random.sample(population, k=k)
    finally:
        random.setstate(state)

# Monkey patch NumPy's random functions to handle int32 overflow and dtype issues
_original_randint = np.random.randint
_original_RandomState_randint = np.random.RandomState.randint

def _safe_randint(low, high=None, size=None, dtype=None):
    """Safe randint that handles int32 overflow and dtype issues"""
    # Handle dtype parameter - convert to proper integer type
    if dtype is not None:
        if np.issubdtype(dtype, np.floating):
            dtype = np.int32  # Convert float dtype to int32
        elif not np.issubdtype(dtype, np.integer):
            dtype = np.int32
    
    if high is None:
        high = low
        low = 0
    
    # Ensure values fit in int32 range
    int32_max = 2**31 - 1
    int32_min = -(2**31)
    
    if isinstance(high, (int, np.integer)) and high > int32_max:
        high = int32_max
    if isinstance(low, (int, np.integer)) and low < int32_min:
        low = 0
    
    # Ensure low < high
    if low >= high:
        high = low + 1
    
    return _original_randint(low, high, size=size, dtype=dtype)

def _safe_RandomState_randint(self, low, high=None, size=None, dtype=None):
    """Safe RandomState.randint that handles dtype issues"""
    # Handle dtype parameter
    if dtype is not None:
        if np.issubdtype(dtype, np.floating):
            dtype = np.int32
        elif not np.issubdtype(dtype, np.integer):
            dtype = np.int32
    
    if high is None:
        high = low
        low = 0
    
    # Ensure values fit in int32 range
    int32_max = 2**31 - 1
    int32_min = -(2**31)
    
    if isinstance(high, (int, np.integer)) and high > int32_max:
        high = int32_max
    if isinstance(low, (int, np.integer)) and low < int32_min:
        low = 0
    
    # Ensure low < high
    if low >= high:
        high = low + 1
    
    return _original_RandomState_randint(self, low, high, size=size, dtype=dtype)

# Replace both np.random.randint and RandomState.randint
np.random.randint = _safe_randint
np.random.RandomState.randint = _safe_RandomState_randint

print("✅ Compatibility patch loaded (NumPy 2.x + Python 3.13)")

TypeError: cannot set 'randint' attribute of immutable type 'numpy.random.mtrand.RandomState'

## 1. Import và Setup

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

from csbdeep.utils import normalize
from stardist import fill_label_holes, random_label_cmap, calculate_extents
from stardist.matching import matching_dataset
from stardist.models import Config2D, StarDist2D

# Apply patch
import csbdeep.utils.utils
csbdeep.utils.utils.choice = _patched_choice
print("✅ Applied compatibility patch")

np.random.seed(42)
lbl_cmap = random_label_cmap()

# Check GPU
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

✅ Applied compatibility patch
TensorFlow: 2.20.0
GPU: []
TensorFlow: 2.20.0
GPU: []


## 2. Load Data

In [2]:
data_dir = Path('my_dataset')

X_train_files = sorted(glob(str(data_dir / 'train' / 'images' / '*')))
Y_train_files = sorted(glob(str(data_dir / 'train' / 'masks' / '*')))
X_val_files = sorted(glob(str(data_dir / 'val' / 'images' / '*')))
Y_val_files = sorted(glob(str(data_dir / 'val' / 'masks' / '*')))

print(f"📊 Dataset size:")
print(f"   Training: {len(X_train_files)} images")
print(f"   Validation: {len(X_val_files)} images")
print(f"   Total: {len(X_train_files) + len(X_val_files)} images")

# Kiểm tra
assert len(X_train_files) > 0, "⚠️ No training images found!"
assert len(X_train_files) == len(Y_train_files), "⚠️ Mismatch in train images/masks!"
assert len(X_val_files) == len(Y_val_files), "⚠️ Mismatch in val images/masks!"

# Đánh giá dataset size
total_images = len(X_train_files) + len(X_val_files)
if total_images >= 150:
    print("\n✅ Dataset size EXCELLENT! Expected AP > 0.85")
elif total_images >= 100:
    print("\n✅ Dataset size GOOD! Expected AP 0.75-0.85")
elif total_images >= 50:
    print("\n⚠️ Dataset size OK. Expected AP 0.65-0.75")
else:
    print("\n⚠️ Dataset size SMALL! Expected AP < 0.70")
    print("   Recommendation: Annotate more images (target: 150-200)")

# Load images
print("\n📥 Loading images...")
X_train = [np.array(Image.open(f)) for f in tqdm(X_train_files, desc="Train images")]
Y_train = [np.array(Image.open(f)) for f in tqdm(Y_train_files, desc="Train masks")]
X_val = [np.array(Image.open(f)) for f in tqdm(X_val_files, desc="Val images")]
Y_val = [np.array(Image.open(f)) for f in tqdm(Y_val_files, desc="Val masks")]

print("\n✅ Data loaded!")

NameError: name 'Path' is not defined

## 3. Preprocessing

In [3]:
n_channel = 1 if X_train[0].ndim == 2 else X_train[0].shape[-1]
print(f"Channels: {n_channel}")

axis_norm = (0,1)
X_train = [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_train, desc="Normalize train")]
X_val = [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(X_val, desc="Normalize val")]

Y_train = [fill_label_holes(y) for y in tqdm(Y_train, desc="Fill holes train")]
Y_val = [fill_label_holes(y) for y in tqdm(Y_val, desc="Fill holes val")]

print("✅ Preprocessing done!")

NameError: name 'X_train' is not defined

## 4. 🔥 AUGMENTATION CẢI TIẾN - MỚI!

**Thay đổi so với version cũ:**
- ✅ Thêm **elastic deformation** (quan trọng cho cells!)
- ✅ Thêm **brightness/contrast** augmentation
- ✅ Tăng **rotation** lên 360 độ
- ✅ Thêm **Gaussian noise**

In [4]:
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage.transform import rotate

def augmenter_strong(x, y):
    """
    Augmentation mạnh mẽ cho StarDist
    
    Áp dụng:
    - Rotation: 0-360 độ
    - Flip: horizontal/vertical
    - Elastic deformation: biến dạng đàn hồi
    - Brightness/Contrast: thay đổi độ sáng/tương phản
    - Gaussian noise: nhiễu Gaussian nhẹ
    """
    # 1. Random rotation (0-360 degrees)
    if np.random.rand() > 0.5:
        angle = np.random.uniform(0, 360)
        x = rotate(x, angle, mode='reflect', preserve_range=True)
        y = rotate(y, angle, order=0, mode='reflect', preserve_range=True).astype(y.dtype)
    
    # 2. Random flip
    if np.random.rand() > 0.5:
        x = np.flip(x, axis=0)
        y = np.flip(y, axis=0)
    if np.random.rand() > 0.5:
        x = np.flip(x, axis=1)
        y = np.flip(y, axis=1)
    
    # 3. Elastic deformation (quan trọng cho cells!)
    if np.random.rand() > 0.5:
        alpha = 30  # Strength of deformation
        sigma = 5   # Smoothness of deformation
        
        shape = x.shape[:2]
        dx = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma) * alpha
        dy = gaussian_filter((np.random.rand(*shape) * 2 - 1), sigma) * alpha
        
        x_coords = np.arange(shape[0])[:, None] + dx
        y_coords = np.arange(shape[1])[None, :] + dy
        
        indices = np.array([x_coords, y_coords])
        
        if x.ndim == 3:  # Color image
            x = np.stack([map_coordinates(x[..., i], indices, order=1, mode='reflect') 
                          for i in range(x.shape[-1])], axis=-1)
        else:  # Grayscale
            x = map_coordinates(x, indices, order=1, mode='reflect')
        
        y = map_coordinates(y, indices, order=0, mode='reflect').astype(y.dtype)
    
    # 4. Brightness adjustment
    if np.random.rand() > 0.5:
        factor = np.random.uniform(0.7, 1.3)
        x = np.clip(x * factor, 0, 1)
    
    # 5. Contrast adjustment
    if np.random.rand() > 0.5:
        factor = np.random.uniform(0.8, 1.2)
        mean = x.mean()
        x = np.clip((x - mean) * factor + mean, 0, 1)
    
    # 6. Gaussian noise
    if np.random.rand() > 0.7:  # 30% chance
        noise = np.random.normal(0, 0.02, x.shape)
        x = np.clip(x + noise, 0, 1)
    
    return x, y

print("✅ Strong augmentation function created!")
print("\n📊 Augmentation details:")
print("   - Rotation: 0-360°")
print("   - Flip: H + V")
print("   - Elastic: α=30, σ=5")
print("   - Brightness: ±30%")
print("   - Contrast: ±20%")
print("   - Gaussian noise: σ=0.02")

✅ Strong augmentation function created!

📊 Augmentation details:
   - Rotation: 0-360°
   - Flip: H + V
   - Elastic: α=30, σ=5
   - Brightness: ±30%
   - Contrast: ±20%
   - Gaussian noise: σ=0.02


## 5. Test Augmentation

In [5]:
# Visualize augmentation examples
test_idx = 0
test_x = X_train[test_idx]
test_y = Y_train[test_idx]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# Original
axes[0, 0].imshow(test_x if test_x.ndim == 3 else test_x, cmap='gray' if test_x.ndim == 2 else None)
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(test_y, cmap=lbl_cmap)
axes[0, 1].set_title('Original Mask')
axes[0, 1].axis('off')

# Augmented examples (5 examples filling remaining 10 slots)
positions = [(0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]

for i in range(1, 6):
    aug_x, aug_y = augmenter_strong(test_x.copy(), test_y.copy())
    
    # Get positions for image and mask
    img_pos = positions[(i-1)*2]
    mask_pos = positions[(i-1)*2 + 1]
    
    # Display augmented image
    axes[img_pos].imshow(aug_x if aug_x.ndim == 3 else aug_x, cmap='gray' if aug_x.ndim == 2 else None)
    axes[img_pos].set_title(f'Augmented {i}')
    axes[img_pos].axis('off')
    
    # Display augmented mask
    axes[mask_pos].imshow(aug_y, cmap=lbl_cmap)
    axes[mask_pos].set_title(f'Augmented Mask {i}')
    axes[mask_pos].axis('off')

plt.tight_layout()
plt.show()

print("✅ Augmentation looks good! Ready to train.")

NameError: name 'X_train' is not defined

## 6. 🔧 CONFIG TỐI ƯU - CẢI TIẾN!

In [6]:
print("🔧 Configuring optimized model...\n")

# Dataset size analysis
n_train = len(X_train)
n_val = len(X_val)

# Auto-adjust parameters based on dataset size
if n_train >= 100:
    train_epochs = 150
    steps_per_epoch = 200
    print("📊 Large dataset detected (≥100 images)")
    print(f"   → epochs={train_epochs}, steps_per_epoch={steps_per_epoch}")
elif n_train >= 50:
    train_epochs = 200
    steps_per_epoch = 150
    print("📊 Medium dataset detected (50-100 images)")
    print(f"   → epochs={train_epochs}, steps_per_epoch={steps_per_epoch}")
else:
    train_epochs = 250
    steps_per_epoch = 100
    print("📊 Small dataset detected (<50 images)")
    print(f"   → epochs={train_epochs}, steps_per_epoch={steps_per_epoch}")
    print("   ⚠️ Consider annotating more data for better results!")

# Model configuration
img_size = X_train[0].shape[:2]
train_patch_size = (256, 256) if min(img_size) < 512 else (512, 512)

print(f"\n🎯 Model configuration:")
print(f"   Image size: {img_size}")
print(f"   Patch size: {train_patch_size}")
print(f"   Channels: {n_channel}")

conf = Config2D(
    n_rays=64,                      # Phù hợp cho cells phức tạp
    grid=(2, 2),                    # Cân bằng tốc độ/chính xác
    use_gpu=False,                  # Python 3.13 compatibility
    n_channel_in=n_channel,
    train_patch_size=train_patch_size,
    
    # Training parameters - OPTIMIZED!
    train_epochs=train_epochs,
    train_steps_per_epoch=steps_per_epoch,
    train_learning_rate=0.0003,     # Conservative learning rate
    train_batch_size=4,             # Batch size 4 works well
    train_reduce_lr={'factor': 0.5, 'patience': 10},  # Learning rate reduction
    
    # Regularization - MỚI!
    train_background_reg=0.0001,    # Background regularization
    train_foreground_only=0.9,      # Focus on foreground
)

print(f"\n✅ Configuration created!")
print(conf)

🔧 Configuring optimized model...



NameError: name 'X_train' is not defined

## 7. Create Model

In [7]:
model_name = 'stardist_my_data_v2_improved'
model_basedir = 'models'

print(f"Creating model: {model_name}...")
model = StarDist2D(conf, name=model_name, basedir=model_basedir)

print(f"\n✅ Model '{model_name}' created!")
print(f"   Location: {model_basedir}/{model_name}/")

Creating model: stardist_my_data_v2_improved...


NameError: name 'StarDist2D' is not defined

## 8. 🚀 TRAINING!

**Expected training time:**
- With GPU: 30-60 minutes
- Without GPU: 2-4 hours

**Expected results:**
- 150+ images: AP > 0.85
- 100-150 images: AP 0.75-0.85
- 50-100 images: AP 0.65-0.75

In [8]:
print("="*60)
print("🚀 STARTING TRAINING WITH STRONG AUGMENTATION")
print("="*60)
print(f"\n📊 Dataset: {n_train} train, {n_val} val")
print(f"⚙️ Config: {train_epochs} epochs × {steps_per_epoch} steps")
print(f"🎨 Augmentation: Strong (rotation+flip+elastic+brightness+contrast+noise)")
print(f"\n⏱️ Estimated time: {'30-60 min' if tf.config.list_physical_devices('GPU') else '2-4 hours'}")
print("="*60 + "\n")

# Training!
history = model.train(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    augmenter=augmenter_strong,  # 🔥 Using strong augmentation!
    epochs=conf.train_epochs,
    steps_per_epoch=conf.train_steps_per_epoch,
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETED!")
print("="*60)

🚀 STARTING TRAINING WITH STRONG AUGMENTATION


NameError: name 'n_train' is not defined

## 9. Optimize Thresholds

In [ ]:
print("🎯 Optimizing thresholds...\n")

model.optimize_thresholds(X_val, Y_val)

print(f"\n✅ Optimized thresholds:")
print(f"   prob_thresh = {model.thresholds.prob:.3f}")
print(f"   nms_thresh = {model.thresholds.nms:.3f}")

## 10. 📊 EVALUATION

**AP (Average Precision) targets:**
- ⭐⭐⭐ Excellent: AP > 0.85
- ⭐⭐ Very Good: AP 0.75-0.85
- ⭐ Good: AP 0.65-0.75
- ⚠️ Needs improvement: AP < 0.65

In [ ]:
print("="*60)
print("📊 EVALUATING MODEL PERFORMANCE")
print("="*60)

# Predict on validation set
print("\nPredicting on validation set...")
Y_val_pred = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(X_val, desc="Predicting")]

# Compute metrics
print("\nComputing metrics...")
stats = [matching_dataset(Y_val, Y_val_pred, thresh=t, show_progress=True) 
         for t in tqdm([0.5, 0.6, 0.7, 0.8, 0.9], desc="Computing metrics")]

print("\n" + "="*60)
print("VALIDATION RESULTS")
print("="*60)
for i, (thresh, stat) in enumerate(zip([0.5, 0.6, 0.7, 0.8, 0.9], stats)):
    print(f"IoU {thresh}: AP = {stat.mean_matched_score:.3f}")

ap_50 = stats[0].mean_matched_score
print(f"\n🎯 OVERALL SCORE (AP@0.5): {ap_50:.3f}")
print("="*60)

# Performance assessment
if ap_50 >= 0.85:
    print("\n🎉🎉🎉 EXCELLENT! AP ≥ 0.85")
    print("✅ Model is ready for production!")
    print("✅ You can now run predictions on all 800 frames!")
elif ap_50 >= 0.75:
    print("\n✅✅ VERY GOOD! AP 0.75-0.85")
    print("✅ Model performance is strong.")
    print("💡 To reach >0.85: Consider adding 20-30 more diverse training images.")
elif ap_50 >= 0.65:
    print("\n✅ GOOD! AP 0.65-0.75")
    print("⚠️ Model is acceptable but can be improved.")
    print("💡 Recommendations:")
    print("   1. Add 50-100 more training images")
    print("   2. Focus on challenging cases (overlapping cells, low contrast)")
    print("   3. Consider increasing n_rays to 96")
else:
    print("\n⚠️ NEEDS IMPROVEMENT! AP < 0.65")
    print("💡 Critical actions needed:")
    print("   1. ✅ CHECK: Are your annotations correct?")
    print("   2. ✅ ADD: At least 100+ more training images")
    print("   3. ✅ DIVERSIFY: Include various lighting, densities, angles")
    print("   4. ✅ TRY: Increase n_rays to 96 or grid to (1,1)")

## 11. Visualize Results

In [ ]:
# Show predictions on random samples
n_samples = min(6, len(X_val))
sample_indices = np.random.choice(len(X_val), n_samples, replace=False)

fig, axes = plt.subplots(n_samples, 4, figsize=(16, 4*n_samples))
if n_samples == 1:
    axes = axes.reshape(1, -1)

for i, idx in enumerate(sample_indices):
    img = X_val[idx]
    mask_true = Y_val[idx]
    mask_pred = Y_val_pred[idx]
    
    # Image
    axes[i, 0].imshow(img if img.ndim == 3 else img, cmap='gray' if img.ndim == 2 else None)
    axes[i, 0].set_title(f"Image {idx}")
    axes[i, 0].axis('off')
    
    # Ground truth
    axes[i, 1].imshow(mask_true, cmap=lbl_cmap)
    n_true = len(np.unique(mask_true)) - 1
    axes[i, 1].set_title(f"Ground Truth ({n_true} cells)")
    axes[i, 1].axis('off')
    
    # Prediction
    axes[i, 2].imshow(mask_pred, cmap=lbl_cmap)
    n_pred = len(np.unique(mask_pred)) - 1
    axes[i, 2].set_title(f"Prediction ({n_pred} cells)")
    axes[i, 2].axis('off')
    
    # Overlay
    from stardist.plot import render_label
    overlay = render_label(mask_pred, img=img if img.ndim == 3 else np.stack([img]*3, axis=-1), alpha=0.3)
    axes[i, 3].imshow(overlay)
    axes[i, 3].set_title(f"Overlay")
    axes[i, 3].axis('off')

plt.tight_layout()
plt.savefig(f'{model_basedir}/{model_name}/validation_results.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Visualization saved to: {model_basedir}/{model_name}/validation_results.png")

## 12. Training History

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Loss
ax1.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# Learning rate
if 'lr' in history.history:
    ax2.plot(history.history['lr'], linewidth=2, color='green')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Learning Rate', fontsize=12)
    ax2.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    ax2.set_yscale('log')
    ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'{model_basedir}/{model_name}/training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Training history saved to: {model_basedir}/{model_name}/training_history.png")

## 13. Save Summary

In [ ]:
# Save training summary
summary = f"""
{'='*60}
TRAINING SUMMARY - {model_name}
{'='*60}

📊 DATASET:
   Training images: {n_train}
   Validation images: {n_val}
   Total images: {n_train + n_val}

⚙️ CONFIGURATION:
   n_rays: {conf.n_rays}
   grid: {conf.grid}
   patch_size: {conf.train_patch_size}
   epochs: {conf.train_epochs}
   steps_per_epoch: {conf.train_steps_per_epoch}
   batch_size: {conf.train_batch_size}
   learning_rate: {conf.train_learning_rate}

🎨 AUGMENTATION:
   Type: Strong (rotation+flip+elastic+brightness+contrast+noise)
   Rotation: 0-360°
   Elastic: α=30, σ=5
   Brightness: ±30%
   Contrast: ±20%

📈 RESULTS:
   AP@0.5: {ap_50:.3f}
   AP@0.6: {stats[1].mean_matched_score:.3f}
   AP@0.7: {stats[2].mean_matched_score:.3f}
   AP@0.8: {stats[3].mean_matched_score:.3f}
   AP@0.9: {stats[4].mean_matched_score:.3f}

🎯 PERFORMANCE:
   {'⭐⭐⭐ EXCELLENT!' if ap_50 >= 0.85 else '⭐⭐ VERY GOOD!' if ap_50 >= 0.75 else '⭐ GOOD' if ap_50 >= 0.65 else '⚠️ NEEDS IMPROVEMENT'}

{'='*60}
"""

print(summary)

# Save to file
with open(f'{model_basedir}/{model_name}/training_summary.txt', 'w') as f:
    f.write(summary)

print(f"\n✅ Summary saved to: {model_basedir}/{model_name}/training_summary.txt")

## 🎯 NEXT STEPS

### If AP ≥ 0.85 (Excellent!):
1. ✅ **Ready for production!**
2. Run `2_prediction_my_data.ipynb` to predict on all 800 frames
3. Export results for analysis

### If AP 0.75-0.85 (Very Good):
1. Model is strong, but can be improved
2. **Optional**: Add 20-30 more diverse training images
3. Focus on difficult cases (overlapping cells, low contrast)
4. Re-train with updated dataset

### If AP 0.65-0.75 (Good):
1. Model is acceptable but needs improvement
2. **Recommended**: Add 50-100 more training images
3. Ensure diversity in training data
4. Consider increasing `n_rays` to 96
5. Re-train

### If AP < 0.65 (Needs Improvement):
1. **Critical**: Check annotation quality
2. **Must**: Add at least 100+ more training images
3. Diversify dataset (lighting, density, angles)
4. Try `n_rays=96` or `grid=(1,1)`
5. Re-train and re-evaluate

---

## 📝 ITERATION WORKFLOW:

```
1. Train → 2. Evaluate → 3. Analyze errors → 4. Add/fix data → 5. Re-train
```

**Remember**: More high-quality annotations = Better model performance!